In [3]:
import numpy as np 
import pandas as pd


In [4]:
data = pd.read_csv('student.csv')
data.head()

,student_id,age,sex,graduated_h_school_type,scholarship_type,additional_work,activity,partner,total_salary,transport,...,preparation_midterm_company,preparation_midterm_time,taking_notes,listenning,discussion_improves_interest,flip_classrom,grade_previous,grade_expected,course_id,grade
0,STUDENT1,2,2,3,3,1,2,2,1,1,...,1,1,3,2,1,2,1,1,1,1
1,STUDENT2,2,2,3,3,1,2,2,1,1,...,1,1,3,2,3,2,2,3,1,1
2,STUDENT3,2,2,2,3,2,2,2,2,4,...,1,1,2,2,1,1,2,2,1,1
3,STUDENT4,1,1,1,3,1,2,1,2,1,...,1,2,3,2,2,1,3,2,1,1
4,STUDENT5,2,2,1,3,2,2,1,3,1,...,2,1,2,2,2,1,2,2,1,1


In [5]:
data_df = data[['age', 'weekly_study_hours', 'attendances_classes', 'taking_notes', 'reading_non_scientific', 'reading_scientific', 'preparation_midterm_company', 'preparation_midterm_time', 'grade']]
data_df.head()

,age,weekly_study_hours,attendances_classes,taking_notes,reading_non_scientific,reading_scientific,preparation_midterm_company,preparation_midterm_time,grade
0,2,3,1,3,2,2,1,1,1
1,2,2,1,3,2,2,1,1,1
2,2,2,1,2,1,2,1,1,1
3,1,3,1,3,1,2,1,2,1
4,2,2,1,2,1,1,2,1,1


In [6]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   age                          145 non-null    int64
 1   weekly_study_hours           145 non-null    int64
 2   attendances_classes          145 non-null    int64
 3   taking_notes                 145 non-null    int64
 4   reading_non_scientific       145 non-null    int64
 5   reading_scientific           145 non-null    int64
 6   preparation_midterm_company  145 non-null    int64
 7   preparation_midterm_time     145 non-null    int64
 8   grade                        145 non-null    int64
dtypes: int64(9)
memory usage: 10.3 KB


In [7]:
def calculate_prior(df, Y):
    classes = sorted(list(df[Y].unique()))
    prior = []
    for i in classes:
        prior.append(len(df[df[Y]==i])/len(df))
    return prior

In [8]:

def calculate_likelihood_gaussian(df, feat_name, feat_val, Y, label):
    #feat = list(df.columns)
    df = df[df[Y]==label]
    mean, std = df[feat_name].mean(), df[feat_name].std()
    p_x_given_y = (1 / (np.sqrt(2 * np.pi) * std)) *  np.exp(-((feat_val-mean)**2 / (2 * std**2 )))
    return p_x_given_y

In [9]:

def calculate_likelihood_categorical(df, feat_name, feat_val, Y,label):
    #feat = list(df.columns)
    df = df[df[Y]==label]
    p_x_given_y = len(df[df[feat_name]==feat_val]) / len(df)
    return p_x_given_y

In [10]:
def naive_bayes(df, X, Y):
    # get feature names
    features = list(df.columns)[:-1]
    print((features))

    # calculate prior
    prior = calculate_prior(df, Y)

    Y_pred = []
    # loop over every data sample
    for x in X:
        # calculate likelihood
        #print(x)
        labels = sorted(list(df[Y].unique()))
        likelihood = [1]*len(labels)
        for j in range(len(labels)):
            for i in range(len(features)):   
                           
             # if features[i]!='weekly_study_hours':
                likelihood[j] *= calculate_likelihood_categorical(df, features[i], x[i], Y, labels[j])
             # else:
               # likelihood[j] *= calculate_likelihood_gaussian(df, features[i], x[i], Y, labels[j])

        
        post_prob = [1]*len(labels)
        for j in range(len(labels)):
            post_prob[j] = likelihood[j] * prior[j]

        Y_pred.append(np.argmax(post_prob))

    return np.array(Y_pred)

In [11]:
#train single layer perceptron and return weight
def train_slp(X, y, num_classes, learning_rate=0.01):
  
    num_features = X.shape[1]
    weights = np.zeros((num_features, num_classes))
    biases = np.zeros(num_classes)
    epochs=0
    #print(X.shape[0])
    #for _ in range(max_epochs):
    prev_error = float('inf')
    while True:
        
        n_e_v=0
        for i in range(X.shape[0]):
            # One-vs-All: create binary labels for the current class
            y_binary = np.where(y[i] == np.arange(num_classes), 1, -1)

            # Forward pass
            scores = np.dot(X[i], weights) + biases
            #print(scores)
            # Compute predicted class
            predicted = np.argmax(scores)
            
            
            e_v= (y[i]- predicted)**2 #[for error calculation]
            n_e_v=n_e_v + e_v
            
            # Update weights and biases
            if predicted != y[i]:
                weights[:, y[i]] += learning_rate * X[i]
                biases[y[i]] += learning_rate

        epochs += 1                           # for showing number of epochs

        error=n_e_v/2
        print(error)                            #calculating error = sum((y_target -y_pred)squre)

        if error >= prev_error:                  # Stop if error is not decreasing
            print("Stopped at epoch", epochs)
            break
        else:
            prev_error = error

    return weights, biases

In [12]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data_df, test_size=.15, random_state=41)

Y_train = train.iloc[:,-1].values
X_train = train.iloc[:,:-1].values
X_test = test.iloc[:,:-1].values
Y_test = test.iloc[:,-1].values
Y_pred_nb = naive_bayes(train, X=X_test, Y="grade")


['age', 'weekly_study_hours', 'attendances_classes', 'taking_notes', 'reading_non_scientific', 'reading_scientific', 'preparation_midterm_company', 'preparation_midterm_time']


In [13]:
# finding Y_pred for slp
def predict_slp(X, weights, biases):
    scores = np.dot(X, weights) + biases
    return np.argmax(scores, axis=1)

In [14]:
# Train the SLP
num_classes = len(np.unique(Y_train))
weights, biases = train_slp(X_train, Y_train, num_classes)
print (weights)

660.5
620.5
585.5
512.5
607.5
Stopped at epoch 5
[[0.5  1.33 1.29 1.34 0.7  1.28 0.85 0.97]
 [1.   1.58 1.55 1.71 0.95 1.56 1.2  1.52]
 [0.5  1.01 1.08 0.74 0.5  0.99 0.65 0.81]
 [0.95 1.98 1.84 1.82 1.   2.02 1.35 1.92]
 [0.8  1.43 1.45 1.38 0.85 1.43 1.1  1.6 ]
 [0.9  1.46 1.56 1.55 0.75 1.57 1.05 1.49]
 [0.75 0.95 0.99 1.04 0.55 0.79 0.6  1.18]
 [0.45 0.81 0.86 0.92 0.5  0.78 0.65 0.95]]


In [15]:
# Make predictions on the testing set
Y_pred_slp = predict_slp(X_test, weights, biases)
print(Y_pred_slp)

[1 2 3 1 1 1 1 5 3 5 2 2 3 3 5 2 7 2 7 1 2 2]


In [16]:
#calculating accuracy for naive bayes
cnt =0
for i in range(len(Y_test)):
  if(Y_test[i]==Y_pred_nb[i]):             
    cnt += 1
accuracy_nb=cnt/len(Y_test)
accuracy_nb

0.045454545454545456

In [17]:
#calculating accuracy for SLP
cnt1 =0
for j in range(len(Y_test)):
  if(Y_test[j]==Y_pred_slp[j]):             
    cnt1 += 1
accuracy_slp=cnt1/len(Y_test)
accuracy_slp

0.2727272727272727

In [18]:
Y_test

array([1, 6, 3, 4, 1, 3, 1, 1, 1, 3, 3, 6, 1, 2, 2, 4, 2, 5, 7, 1, 1, 5])

In [19]:
Y_pred_nb

array([2, 0, 5, 2, 5, 4, 1, 5, 3, 1, 6, 2, 7, 4, 5, 1, 7, 2, 1, 2, 2, 2])

In [20]:
Y_pred_slp

array([1, 2, 3, 1, 1, 1, 1, 5, 3, 5, 2, 2, 3, 3, 5, 2, 7, 2, 7, 1, 2, 2])